In [1]:
import glob
import pandas as pd
import plotly.graph_objects as go
from ipywidgets import interact
from editolido.geoindex import GeoGridIndex
from editolidoProxy import (
    load_ofp_route,
    get_nearest_wmo,
    get_nearby_wmo,
    basemap
)
wmo_grid = GeoGridIndex()
wmo_grid.load()
wmo_stations = (dict(name=item[0], lat=item[1], lng=item[2]) for sublist in wmo_grid.data.values() for item in sublist)
df = pd.DataFrame(wmo_stations)


@interact
def plot(file=glob.glob('ofp/*.route.json')):
    route = load_ofp_route(file)
    latmin = min([p.latitude for p in route]) - 1
    latmax = max([p.latitude for p in route]) + 1
    lngmin = min([p.longitude for p in route]) - 1
    lngmax= max([p.longitude for p in route]) + 1
    bb = df[(df['lat'] > latmin) & (df['lat'] < latmax) & (df['lng'] > lngmin) & (df['lng'] < lngmax)]
    nearest_wmo = get_nearest_wmo(route)
    nearby_wmo = get_nearby_wmo(route)
    fig = basemap(route, nearby_wmo=nearby_wmo, nearest_wmo=nearest_wmo, title='Route and nearest WMO')
    fig.add_trace(
        go.Scattergeo(
            lon = bb['lng'],
            lat = bb['lat'],
            text = bb['name'],
            mode = 'markers',
            visible='legendonly',
            name='wmo stations',
            marker=dict(size=4)
        ))
    return fig

interactive(children=(Dropdown(description='file', options=('ofp/BST_AF650 _LFPG_MMUN_20180304_1355z.route.jso…